In [2]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

    # #groq = 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

In [ ]:
%pip install langchain-groq
%pip install -qU langchain-groq
%pip install -U langgraph
%pip install spotipy


In [81]:
from typing import List, TypedDict, Annotated, Any
from langgraph.graph import END, START, StateGraph, MessagesState
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate 

class SpotifyState(MessagesState):  # Inherit both
    all_playlists: List[dict]  # Store playlist dictionaries (name and ID)
    selected_playlist_id: str  # ID of the selected playlist
    tool_result: dict  # Result of the last tool call
    user_input: str  # User's natural language input
    song_name: str # Store the song name
    artist_name: str # Store the artist name

    def __init__(self, user_input=None, **kwargs):
        super().__init__(**kwargs)
        self.user_input = user_input
        self.all_playlists = [] # Initialize all_playlists
        self.selected_playlist_id = None
        self.tool_result = None
        self.song_name = None
        self.artist_name = None

In [82]:
assistant_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful assistant that helps users manage their Spotify playlists."),
    HumanMessage(content="The user's request is: {user_input}"),
    HumanMessage(content="""Based on the user's request, decide which tool to use (if any).  Respond with a json object like this:

```json
{{
  "tool": "<tool_name>",  // e.g., "show_playlist_tool", "create_playlist_tool", "search_track_tool", "add_track_tool", or null if no tool is needed
  "arguments": {{
    "<argument1_name>": "<argument1_value>", // e.g., "song_name": "Mistletoe", "artist_name": "Justin Bieber"
    "<argument2_name>": "<argument2_value>",
    // ... other arguments
  }},
  "response": "<response_to_user>" // A response to show to the user
}}
                 """),
MessagesPlaceholder("messages")  # Include message history
])

In [83]:
# Spotify Authentication
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from langchain_groq import ChatGroq
from langchain.tools import Tool



SCOPE = "playlist-modify-public playlist-read-private playlist-modify-private"
sp_oauth = SpotifyOAuth(
    client_id="4a8fdc631a704303ab465a07d5141a61",  # Replace with your credentials
    client_secret="d649ba5407504a928eb2b94b49fa234d",
    redirect_uri="http://localhost:8888/callback",
    scope=SCOPE,
)
sp = spotipy.Spotify(auth_manager=sp_oauth)

# LLM Setup
llm = ChatGroq(
    model="mixtral-8x7b-32768",  # Or your preferred model
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [88]:
print(type(sp))

<class 'spotipy.client.Spotify'>


In [84]:
from langchain.tools import Tool

def show_playlist_tool(state: SpotifyState):
    all_playlists=state["all_playlists"]
    """
    Tool to show the user's playlists.

    Args:
        state (SpotifyState): The current state.

    Returns:
        SpotifyState: The updated state with the list of playlists in `state.all_playlists`.
    """
    results = sp.current_user_playlists()
    state.all_playlists = [{"name": item['name'], "id": item['id']} for item in results['items']]
    all_playlists=state["all_playlists"]
    return {"all_playlists": state.all_playlists}
def create_playlist_tool(state: SpotifyState, playlist_name: str):
    all_playlists=state["all_playlists"]
    """
    Tool to create a new playlist.

    Args:
        state (SpotifyState): The current state.
        playlist_name (str): The name of the new playlist.

    Returns:
        SpotifyState: The updated state with the newly created playlist in `state.all_playlists`.
    """
    new_playlist = sp.user_playlist_create(sp.me()['id'], playlist_name)
    state.all_playlists.append({"name": playlist_name, "id": new_playlist['id']})
    return {"all_playlist": state.all_playlists}

def search_track_tool(song_name: str, artist_name: str):
    """
    Tool to search for a track.

    Args:
        song_name (str): The name of the song.
        artist_name (str): The name of the artist.

    Returns:
        dict: A dictionary containing track details (URI, name, artist), or None if not found.
    """
    results = sp.search(q=f"track:{song_name} artist:{artist_name}", type='track')
    if results['tracks']['items']:
        track = results['tracks']['items'][0]
        return {
            "track_uri": track['uri'],
            "track_name": track['name'],
            "track_artist": track['artists'][0]['name']
        }
    return None

def add_track_tool(state: SpotifyState):
    selected_playlist_id= state["selected_playlist_id"]
    """
    Tool to add a track to the selected playlist.

    Args:
        state (SpotifyState): The current state.

    Returns:
        SpotifyState: The updated state.
    """
    if state.tool_result and state.selected_playlist_id:
        sp.playlist_add_items(state.selected_playlist_id, [state.tool_result['track_uri']])
        return state
    return state  # Return state even if adding fails (handle in agent)

tools = [
    Tool(
        name="show_playlist_tool",
        func=show_playlist_tool,
        description="Shows the user's playlists.",
    ),
    Tool(
        name="create_playlist_tool",
        func=create_playlist_tool,
        description="Creates a new playlist. Requires the playlist name as an argument.",
    ),
    Tool(
        name="search_track_tool",
        func=search_track_tool,
        description="Searches for a track. Requires the song name and artist name as arguments.",
    ),
    Tool(
        name="add_track_tool",
        func=add_track_tool,
        description="Adds a track to the selected playlist. Requires the track details from the search_track_tool and the selected playlist ID.",
    ),
]

llm_with_tools = llm.bind_tools(tools, parallel_tool_calls=False)


In [77]:
#approach2 for tooldefs: 

from langchain.tools import Tool

def show_playlist_tool(state: SpotifyState):
    """
    Tool to show the user's playlists.

    Args:
        state (SpotifyState): The current state.


    Returns:
        An update in the state values of "all_playlist"
    """
    results = sp.current_user_playlists()
    return {
        "all_playlists": [{"name": item['name'], "id": item['id']} 
                         for item in results['items']]
    }
def create_playlist_tool(state: SpotifyState, playlist_name: str):
    """
    Tool to create a new playlist.

    Args:

        playlist_name (str): The name of the new playlist.
        state (SpotifyState): The current state.


    Returns:
        SpotifyState: The updated state with the newly created playlist in `state.all_playlists`.
    """
    return {
        "all_playlists": [{"name": playlist_name, "id": playlist_name['id']}]
    }

def search_track_tool(song_name: str, artist_name: str, state: SpotifyState):
    """
    Tool to search for a track.

    Args:
        song_name (str): The name of the song.
        artist_name (str): The name of the artist.

    Returns:
        dict: A dictionary containing track details (URI, name, artist), or None if not found.
    """
    results = sp.search(q=f"track:{song_name} artist:{artist_name}", type='track')
    if results['tracks']['items']:
        track = results['tracks']['items'][0]
        return {
            "track_uri": track['uri'],
            "track_name": track['name'],
            "track_artist": track['artists'][0]['name']
        }
    return None

def add_track_tool(playlist_id: str, track_uri : str,state: SpotifyState):
    """
    Tool to add a track to the selected playlist.

    Args:
        playlist_id (str) : id of the playlist
        track_uri: uri of the track 

    Returns:
        SpotifyState: The updated state.
    """
    sp.playlist_add_items(playlist_id, [track_uri])
    return {"status": "success"}

tools = [
    Tool(
        name="show_playlist_tool",
        func=show_playlist_tool,
        description="Shows the user's playlists.",
    ),
    Tool(
        name="create_playlist_tool",
        func=create_playlist_tool,
        description="Creates a new playlist. Requires the playlist name as an argument.",
    ),
    Tool(
        name="search_track_tool",
        func=search_track_tool,
        description="Searches for a track. Requires the song name and artist name as arguments.",
    ),
    Tool(
        name="add_track_tool",
        func=add_track_tool,
        description="Adds a track to the selected playlist. Requires the track details from the search_track_tool and the selected playlist ID.",
    ),
]

llm_with_tools = llm.bind_tools(tools, parallel_tool_calls=False)


In [ ]:
#suggestions to define how the state is updated
#after each tool call 
def update_state(state: SpotifyState, tool_result: dict):
    if "all_playlists" in tool_result:
        state.all_playlists = tool_result["all_playlists"]
    return state

In [78]:
# new approach for ai_agent
#deepseek
import json
def ai_agent(state: SpotifyState):
    messages = state.messages
    prompt = assistant_prompt.format(user_input=state.user_input, messages=messages)
    response = llm_with_tools.invoke(prompt)
    state.messages.append(HumanMessage(content=state.user_input))
    state.messages.append(response)

    try:
        tool_calls = response.additional_kwargs.get("tool_calls", [])

        if tool_calls:
            for tool_call in tool_calls:
                tool_name = tool_call['function']['name']
                arguments = json.loads(tool_call['function']['arguments'])
                tool = next((t for t in tools if t.name == tool_name), None)

                if tool:
                    # Handle each tool type
                    if tool.name == "show_playlist_tool":
                        tool_result = tool.func()
                        state.all_playlists = tool_result["all_playlists"]  # Update state
                        state.messages.append(HumanMessage(
                            content=f"Playlists loaded: {len(state.all_playlists)} found"
                        ))

                    elif tool.name == "create_playlist_tool":
                        tool_result = tool.func(arguments.get("playlist_name"))
                        new_playlist = tool_result["new_playlist"]
                        state.all_playlists.append(new_playlist)  # Append to existing list
                        state.messages.append(HumanMessage(
                            content=f"Created playlist: {new_playlist['name']}"
                        ))

                    elif tool.name == "search_track_tool":
                        song = arguments.get("song_name")
                        artist = arguments.get("artist_name")
                        tool_result = tool.func(song, artist)
                        state.tool_result = tool_result  # Store result in state
                        state.messages.append(HumanMessage(
                            content=f"Found track: {tool_result['track_name']}" if tool_result 
                            else "Track not found"
                        ))

                    elif tool.name == "add_track_tool":
                        if state.selected_playlist_id and state.tool_result:
                            tool_result = tool.func(
                                state.selected_playlist_id,
                                state.tool_result['track_uri']
                            )
                            state.messages.append(HumanMessage(
                                content="Track added successfully!"
                            ))
                        else:
                            state.messages.append(HumanMessage(
                                content="Need to select a playlist first!"
                            ))
    except (json.JSONDecodeError, KeyError) as e:
            state['messages'].append(HumanMessage(content=f"Error processing response: {e}"))
            return state
            # ... rest of error handling ...

    return state

In [ ]:
import json
def ai_agent(state: SpotifyState):
    messages = state['messages']  # Correct way to access messages
    prompt = assistant_prompt.format(user_input=state['user_input'], messages=messages)
    response = llm_with_tools.invoke(prompt)
    messages.append(HumanMessage(content=state["user_input"]))  # Correct way to add messages
    messages.append(response)

    try:
        json_response = json.loads(response.content)
        tool_name = json_response.get("tool")
        arguments = json_response.get("arguments", {})
        response_to_user = json_response.get("response")

        if tool_name:
            tool = next((t for t in tools if t.name == tool_name), None)
            if tool:
                if tool.name == "search_track_tool":
                  state.song_name = arguments.get("song_name")
                  state.artist_name = arguments.get("artist_name")
                  tool_result = tool.func(arguments.get("song_name"), arguments.get("artist_name"))
                  state.tool_result = tool_result # Store the result in the state
                  if tool_result:
                    messages.append(HumanMessage(content=f"Tool {tool_name} returned: {tool_result}")) # Use add_message
                  else:
                    messages.append(HumanMessage(content=f"Tool {tool_name} returned None. Track not found")) # Use add_message

                elif tool.name == "create_playlist_tool":
                  state = tool.func(state, arguments.get("playlist_name"))
                  messages.append(HumanMessage(content=f"Tool {tool_name} created playlist {arguments.get('playlist_name')}"))# Use add_message
                elif tool.name == "show_playlist_tool":
                  state = tool.func(state)
                  messages.append(HumanMessage(content=f"Tool {tool_name} returned playlists: {state.all_playlists}"))# Use add_message
                elif tool.name == "add_track_tool":
                  state = tool.func(state)
                  messages.append(HumanMessage(content=f"Tool {tool_name} added track"))# Use add_message


            else:
                messages.append(HumanMessage(content="Invalid tool name.")) # Use add_message
        if response_to_user:
          messages.append(HumanMessage(content=response_to_user)) # Use add_message

        return {"messages": messages}  # Return updated state

    except (json.JSONDecodeError, KeyError) as e:
        messages.append(HumanMessage(content=f"Error processing response: {e}")) # Use add_message
        return {"messages": messages}

In [85]:
#APPROACH 2: 
def ai_agent(state: SpotifyState):
    messages = state['messages']
    prompt = assistant_prompt.format(user_input=state['user_input'], messages=messages)
    response = llm_with_tools.invoke(prompt)
    state['messages'].append(HumanMessage(content=state['user_input']))
    state['messages'].append(response)

    try:
        tool_calls = response.additional_kwargs.get("tool_calls", [])  # Get tool calls

        if tool_calls:  # Check if tool calls exist
            for tool_call in tool_calls: # Iterate over the tools
                tool_name = tool_call['function']['name']
                arguments = json.loads(tool_call['function']['arguments'])  # Parse JSON args
                tool = next((t for t in tools if t.name == tool_name), None)

                if tool:
                    if tool.name == "search_track_tool":
                        state.song_name = arguments.get("song_name")
                        state.artist_name = arguments.get("artist_name")
                        tool_result = tool.func(arguments.get("song_name"), arguments.get("artist_name"))
                        state.tool_result = tool_result
                        if tool_result:
                            state['messages'].append(HumanMessage(content=f"Tool {tool_name} returned: {tool_result}"))
                        else:
                            state['messages'].append(HumanMessage(content=f"Tool {tool_name} returned None. Track not found"))

                    elif tool.name == "create_playlist_tool":
                        state = tool.func(state, arguments.get("playlist_name"))
                        state['messages'].append(HumanMessage(content=f"Tool {tool_name} created playlist {arguments.get('playlist_name')}"))
                    elif tool.name == "show_playlist_tool":
                        state = tool.func(state)
                        state['messages'].append(HumanMessage(content=f"Tool {tool_name} returned playlists: {state.all_playlists}"))
                    elif tool.name == "add_track_tool":
                        state = tool.func(state)
                        state['messages'].append(HumanMessage(content=f"Tool {tool_name} added track"))

                else:
                    state['messages'].append(HumanMessage(content="Invalid tool name."))
        else:
            # Handle cases where LLM doesn't call a tool but returns a text response
            try:
              json_response = json.loads(response.content)
              response_to_user = json_response.get("response")
              if response_to_user:
                state['messages'].append(HumanMessage(content=response_to_user))
            except json.JSONDecodeError:
                # Handle cases where the LLM response is not valid JSON
                state['messages'].append(HumanMessage(content=response.content)) # Just add the raw response

        return state

    except (json.JSONDecodeError, KeyError) as e:
        state['messages'].append(HumanMessage(content=f"Error processing response: {e}"))
        return state

In [86]:
builder = StateGraph(SpotifyState)
builder.add_node("ai_agent", ai_agent)
builder.set_entry_point("ai_agent")
graph = builder.compile()



In [87]:
user_input = "Hello, help me add a song to my playlists"  # Example user input
state = SpotifyState(user_input=user_input)
result = graph.invoke(state)                   

APIConnectionError: Connection error.

In [70]:
print(result['messages'][-1])

content="Error processing response: 'all_playlists'" additional_kwargs={} response_metadata={} id='11021337-b049-46dd-8be0-ddf7a155ceff'
